# Specifying Custom Gradients
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).


### Solving a Simple Mathematical Program
In this case, we are optimizing a simple 2-variable cost function `x_0 + 2 * x_1`, where both variables are constrained to be in the range `[0..1]`.

Without any additional constraints, we can find the optimal solution to be `x_0 = x_1 = 0`, which yields a cost of `0`.

By adding a constraint that the solution cannot lie on a circle centered at the origin with radius 0.5, we instead get an optimal solution of `x_0 = 0.5` and `x_1 = 0.0`, which yields a cost of `0.5`.

In [178]:
import numpy as np
from pydrake.solvers import MathematicalProgram, Solve

prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(x[0] + 2.0 * x[1])
prog.AddBoundingBoxConstraint(0.0, 1.0, x)
prog.AddConstraint(x[0]**2 + x[1]**2 >= 0.5**2)

result = Solve(prog, np.array([1.0, 1.0]))
print("Success? ", result.is_success())
print(result.get_solution_result())
print("x*= ", result.GetSolution(x))

Success?  True
SolutionResult.kSolutionFound
x*=  [0.50000004 0.        ]


### Providing Your Own Gradients
For simple expressions like these, AutoDiff already took care of calculating all the gradients for optimization.

However, assume you have a more complex function that cannot accept these `AutoDiffXd` object that get created by Drake. In this case, you can manually specify gradients yourself, for both cost and constraint expressions.

In [179]:
from pydrake.autodiffutils import AutoDiffXd, ExtractValue

def cost_expression(x):
    if x.dtype == float:
        # The cost expression with floats.
        return x[0] + 2.0 * x[1]
    else:
        # Extract the values from the AutoDiffXd object
        x_value = ExtractValue(x)

        # Compute the cost and gradient
        # NOTE: x_value is a column vector so we specify two indices to avoid warnings
        cost = x_value[0,0] + 2.0 * x_value[1,0]
        gradient = np.array([1.0, 2.0])

        # Return the new AutoDiffXd object with the cost and gradient
        return AutoDiffXd(cost, gradient)

def constraint_expression(x):
    if x.dtype == float:
        # The constraint expression with floats
        return x[0]**2 + x[1]**2
    else:
        # Extract the values from the AutoDiffXd object
        x_value = ExtractValue(x)

        # Compute the constraint and gradient
        # NOTE: x_value is a column vector so we specify two indices to avoid warnings
        constraint = x_value[0, 0]**2 + x_value[1, 0]**2
        gradient = np.array([2.0 * x_value[0, 0], 2.0 * x_value[1, 0]])

        # Return the new AutoDiffXd object with the constraint and gradient
        # NOTE: In this case, we have to wrap the expression in a numpy array
        return np.array([AutoDiffXd(constraint, gradient)])

prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(cost_expression, x)
prog.AddBoundingBoxConstraint(0.0, 1.0, x)

# For an expression-based constraint, the lower and upper bounds must also be numpy arrays
lb = 0.5**2 * np.ones(1)
ub = np.inf * np.ones(1)
prog.AddConstraint(constraint_expression, lb, ub, x)

result = Solve(prog, np.array([1.0, 1.0]))
print("Success? ", result.is_success())
print(result.get_solution_result())
print("x*= ", result.GetSolution(x))

Success?  True
SolutionResult.kSolutionFound
x*=  [0.50000007 0.        ]


### Calculating Gradients using Finite Differencing
In other cases, you may not have analytical gradients to provide to the optimizer.

You can always rely on *finite differencing*, or numerical methods, to approximate the gradients by evaluating expressions at places slightly perturbed from the nominal value.

Please note that you should only rely on finite differencing when absolutely necessary, as this can drastically slow down the optimization process.

In [180]:
from pydrake.math import ComputeNumericalGradient, NumericalGradientMethod, NumericalGradientOption

def cost_fn(x):
    # This must return a numpy array to be compatible with numerical gradient extraction
    return np.array([x[0] + 2.0 * x[1]])

def cost_expression(x):
    if x.dtype == float:
        # The cost expression with floats.
        return x[0] + 2.0 * x[1]
    else:
        # Extract the values from the AutoDiffXd object
        x_value = ExtractValue(x)

        # Compute the cost and approximate gradient
        # We are using forward differencing here as the `option` argument.
        cost = cost_fn(x_value)
        gradient = ComputeNumericalGradient(cost_fn, x_value, option=NumericalGradientOption(NumericalGradientMethod.kForward))

        # Return the new AutoDiffXd object with the cost and gradient
        return AutoDiffXd(cost[0,0], gradient[0,:])

def constraint_fn(x):
    # This must return a numpy array to be compatible with numerical gradient extraction
    return np.array([x[0]**2 + x[1]**2])

def constraint_expression(x):
    if x.dtype == float:
        # The constraint expression with floats
        return x[0]**2 + x[1]**2
    else:
        # Extract the values from the AutoDiffXd object
        x_value = ExtractValue(x)

        # Compute the constraint and approximate gradient
        # We are using central differencing here as the `option` argument.
        constraint = constraint_fn(x_value)
        gradient = ComputeNumericalGradient(constraint_fn, x_value, option=NumericalGradientOption(NumericalGradientMethod.kCentral))
        
        # Return the new AutoDiffXd object with the constraint and gradient
        return np.array([AutoDiffXd(constraint[0,0], gradient[0,:])])
                                  
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(cost_expression, x)
prog.AddBoundingBoxConstraint(0.0, 1.0, x)

# For an expression-based constraint, the lower and upper bounds must also be numpy arrays
lb = 0.5**2 * np.ones(1)
ub = np.inf * np.ones(1)
prog.AddConstraint(constraint_expression, lb, ub, x)

result = Solve(prog, np.array([1.0, 1.0]))
print("Success? ", result.is_success())
print(result.get_solution_result())
print("x*= ", result.GetSolution(x))

Success?  True
SolutionResult.kSolutionFound
x*=  [0.50000007 0.        ]
